In [ ]:
# | hide
from onprem.core import *

# Use Prompts to Solve Problems

This notebook shows various examples of using **[OnPrem.LLM](https://github.com/amaiya/onprem)** to solve different tasks.

## Setup the `LLM` instance

In this notebook, we will use a model called **[Mistral-7B-Instruct-v0.1](https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF)**.  When selecting a model, it is important to inspect the model's home page and identify the correct prompt format.  The prompt format for this model is [located here](https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF#prompt-template-mistral), and we will supply it directly to the `LLM` constructor along with the URL to the specific model file we want (i.e., *mistral-7b-instruct-v0.1.Q4_K_M.gguf*).  We will offload layers to our GPU(s) to speed up inference using the `n_gpu_layers` parameter. (For more information on GPU acceleration, see [here](https://amaiya.github.io/onprem/#speeding-up-inference-using-a-gpu).) For the purposes of this notebook, we also supply `temperature=0` so that there is no variability in outputs.  You can increase this value for more creativity in the outputs.

In [ ]:
# | notest
from onprem import LLM
import os

llm = LLM(model_url='https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf', 
          prompt_template= "<s>[INST] {prompt} [/INST]",
          n_gpu_layers=33,
          temperature=0)

## Information Extraction

This is an example of *zero-shot prompting*:

In [ ]:
# | notest
prompt = """Extract the names of people in the supplied sentences. Separate the names with commas.
[Sentence]: I like Cillian Murphy's acting. Florence Pugh is great, too.
[People]:"""

saved_output = llm.prompt(prompt, stop=['\n\n'])

 Cillian Murphy, Florence Pugh

A more complicated example of **Information Extraction** using *few-shot prompting*:

In [ ]:
# | notest

prompt = """ Extract the Name, Current Position, and Current Company from each piece of Text. 

Text: Alan F. Estevez serves as the Under Secretary of Commerce for Industry and Security.  As Under Secretary, Mr. Estevez leads
the Bureau of Industry and Security, which advances U.S. national security, foreign policy, and economic objectives by ensuring an
effective export control and treaty compliance system and promoting U.S. strategic technology leadership.
A: Name:  Alan F. Estevez | Current Position: Under Secretary | Current Company: Bureau of Industry and Security

Text: Pichai Sundararajan (born June 10, 1972[3][4][5]), better known as Sundar Pichai (/ˈsʊndɑːr pɪˈtʃaɪ/), is an Indian-born American
business executive.[6][7] He is the chief executive officer (CEO) of Alphabet Inc. and its subsidiary Google.[8]
A: Name:   Sundar Pichai | Current Position: CEO | Current Company: Google

Text: Norton Allan Schwartz (born December 14, 1951)[1] is a retired United States Air Force general[2] who served as the 19th Chief of Staff of the 
Air Force from August 12, 2008, until his retirement in 2012.[3] He previously served as commander, United States Transportation Command from 
September 2005 to August 2008. He is currently the president of the Institute for Defense Analyses, serving since January 2, 2020.[4]
A:"""
saved_output = llm.prompt(prompt, stop=['\n\n'])

 Name: Norton Allan Schwartz | Current Position: President | Current Company: Institute for Defense Analyses

## Grammar Correction

In [ ]:
# | notest

prompt = """Correct the grammar and spelling in the supplied sentences.  Here are some examples.
[Sentence]:
I love goin to the beach.
[Correction]: I love going to the beach.
[Sentence]:
Let me hav it!
[Correction]: Let me have it!
[Sentence]:
It have too many drawbacks.
[Correction]: It has too many drawbacks.
[Sentence]:
I do not wan to go
[Correction]:"""
saved_output = llm.prompt(prompt, stop=['\n\n'])

 I do not want to go.

## Classification

In [ ]:
# | notest

prompt = """Classify each sentence as either positive, negative, or neutral.  Here are some examples.
[Sentence]: I love going to the beach.
[[Classification]: Positive
[Sentence]: It is 10am right now.
[Classification]: Neutral
[Sentence]: I just got fired from my job.
[Classification]: Negative
[Sentence]: The reactivity of  your team has been amazing, thanks!
[Classification]:"""

saved_output = llm.prompt(prompt, stop=['\n\n'])

 Positive

## Paraphrasing

In [ ]:
# | notest

prompt = """Paraphrase the following text delimited by triple backticks using a single sentence. 
```After a war lasting 20 years, following the decision taken first by President Trump and then by President Biden to withdraw American troops, Kabul, the capital of Afghanistan, fell within a few hours to the Taliban, without resistance.```
"""
saved_output = llm.prompt(prompt)

 Following the withdrawal of American troops, as decided by Presidents Trump and Biden, Kabul, the capital of Afghanistan, was captured by the Taliban in just a few hours without any opposition.

## Few-Shot Answer Extraction

In [ ]:
# | notest

prompt = """ Compelte the answer:
[Question]: When was NLP Cloud founded?
[Context]: NLP Cloud was founded in 2021 when the team realized there was no easy way to reliably leverage Natural Language Processing in production.
[Answer]: 2021

[Question]:  What did NLP Cloud develop?
[Context]: NLP Cloud developed their API by mid-2020 and they added many pre-trained open-source models since then.
[Answer]: API

[Question]: When can plans be stopped?
[Context]: All plans can be stopped anytime. You only pay for the time you used the service. In case of a downgrade, you will get a discount on your next invoice.
[Answer]: Anytime

[Question]: Which plan is recommended for GPT-J?
[Context]: The main challenge with GPT-J is memory consumption. Using a GPU plan is recommended.
[Answer]:"""
saved_output = llm.prompt(prompt, stop=['\n\n'])

 GPU plan

## Generating Product Descriptions

In [ ]:
# | notest

prompt = """Generate a short Sentence from the Keywords. Here are some examples.
[Keywords]: shoes, women, $59
[Sentence]: Beautiful shoes for women at the price of $59.

[Keywords]: trousers, men, $69
[Sentence]: Modern trousers for men, for $69 only.

[Keywords]: gloves, winter, $19
[Sentence]:  Amazingly hot gloves for cold winters, at $19.

[Keywords]:  t-shirt, men, $39
[Sentence]:"""
saved_output = llm.prompt(prompt, stop=['\n\n'])

 Stylish t-shirts for men at an affordable price of $39.

## Tweet Generation

In [ ]:
# | notest
prompt = """Generate a tweet based on the supplied Keyword. Here are some examples.
[Keyword]:
markets
[Tweet]:
Take feedback from nature and markets, not from people
###
[Keyword]:
children
[Tweet]:
Maybe we die so we can come back as children.
###
[Keyword]:
startups
[Tweet]: 
Startups should not worry about how to put out fires, they should worry about how to start them.
###
[Keyword]:
climate change
[Tweet]:"""

saved_output = llm.prompt(prompt)

 Climate change is real and it's time for action. Let's work together to reduce our carbon footprint and protect our planet for future generations. #ClimateActionNow

## Text Generation

(FYI: The [Zephyr-7B](https://huggingface.co/TheBloke/zephyr-7B-beta-GGUF) model seems to be a bit better at story-telling than Mistral-7B.)

In [ ]:
# | notest
prompt = """Generate an email introducing Tesla to shareholders."""
saved_output = llm.prompt(prompt)

 Subject: Introducing Tesla to Our Shareholders

Dear Valued Shareholders,

We are excited to announce that we have recently acquired a stake in Tesla, Inc., a leading electric vehicle and clean energy company. As one of our valued shareholders, we believe this investment aligns with our commitment to sustainable practices and long-term growth opportunities.

Tesla is at the forefront of innovation in the automotive industry, with a mission to accelerate the world's transition to sustainable energy. Their electric vehicles are not only eco-friendly but also offer cutting-edge technology and exceptional performance. In addition to their automotive business, Tesla has made significant strides in the development of renewable energy solutions, including solar panels and battery storage systems.

We believe that our investment in Tesla will provide us with exposure to a rapidly growing market and the opportunity to generate attractive returns for our shareholders. We are confident in Tesla'

## Talk to Your Documents


In [ ]:
# | notest
llm.ingest("./sample_data/")

Creating new vectorstore at /home/amaiya/onprem_data/vectordb
Loading documents from ./sample_data/


Loading new documents: 100%|██████████████████████| 3/3 [00:00<00:00, 22.00it/s]


Loaded 12 new documents from ./sample_data/
Split into 153 chunks of text (max. 500 chars each)
Creating embeddings. May take some minutes...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.78s/it]

Ingestion complete! You can now query your documents using the LLM.ask or LLM.chat methods


In [ ]:
# | notest

result = llm.ask("What is ktrain?")

 ktrain is a low-code library for augmented machine learning that automates or semi-automates the full machine learning workflow from curating and preprocessing inputs to training, tuning, troubleshooting, and applying models. It is designed to help further democratize machine learning by enabling beginners and domain experts with minimal programming or data science experience.

**Pro-Tip**: You can try different models or re-phrase the question, which may provide better performance for certain tasks.  For instance, you can use the default [Wizard-Vicuna-7B-Uncensored](https://huggingface.co/TheBloke/Wizard-Vicuna-7B-Uncensored-GGUF) model, which can also solve the above tasks, as shown in this example [Google Colab notebook](https://colab.research.google.com/drive/1LVeacsQ9dmE1BVzwR3eTLukpeRIMmUqi?usp=sharing) of **OnPrem.LLM**.